In [1]:
using Pkg
Pkg.activate("/home/fedflorio/master_thesis")

  Activating project at `~/master_thesis`


In [2]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays;

Precompiling MatrixProductBP
  ✓ MatrixProductBP
  1 dependency successfully precompiled in 24 seconds. 119 already precompiled.


In [3]:
# T = 15
# N = 4
# seed = 4

# c = 4
# gg = erdos_renyi(N, c/N; seed)
# g = IndexedGraph(gg)

# λ_unif = 0.8
# ρ_unif = 0.6
# λ = zeros(N,N)
# for i in CartesianIndices(λ)
#     if !iszero(g.A[i])
#         # λ[i] = rand()
#         λ[i] = λ_unif
#     end
# end
# λ = sparse(λ)
# # ρ = rand(N)
# ρ = fill(ρ_unif,N)
# γ = 0.8

T = 7
N = 2
seed = 6

A = [0 1; 1 0]
g = IndexedGraph(A)

λ_unif = 0.7
ρ_unif = 0.6
λ = sparse(λ_unif .* A)
# λ = sparse([0 1e-12; λ_unif 0])
ρ = fill(ρ_unif, N)
γ = 0.5
;

In [4]:
sis = SIS_heterogeneous(λ, ρ, T; γ);
bp_obs = mpbp(sis);

In [5]:
g.A

2×2 SparseMatrixCSC{Int64, Int64} with 2 stored entries:
 ⋅  1
 1  ⋅

In [6]:
obs_times = collect(0:T)
nobs = floor(Int, N * length(obs_times) * 1.0)
obs_fraction = nobs / N
rng = MersenneTwister(seed)
X, observed = draw_node_observations!(bp_obs, nobs, times = obs_times .+ 1, softinf=Inf; rng);

In [7]:
X

2×8 Matrix{Int64}:
 2  2  2  2  2  1  2  1
 1  1  2  1  2  2  2  1

In [8]:
λinit = 0.5
ρinit = 0.5

A_complete = ones(N,N) - I
g_complete = IndexedGraph(A_complete)
λ_complete = sparse(λinit.*A_complete)
ρ_complete = fill(ρinit, N)

sis_inf = SIS_heterogeneous(g_complete, λ_complete, ρ_complete, T; γ, ϕ=deepcopy(bp_obs.ϕ))
bp_inf = mpbp(sis_inf);

In [9]:
svd_trunc = TruncBond(5)

iters, cb = inference_parameters!(bp_inf, method=31, maxiter=200, λstep=0.01, ρstep=0.01);

Running Gradient Ascent: iter 199    Time: 0:03:46
  Δ:              0.6699999999999999


In [10]:
@show cb.data

cb.data = PARAMS{Float64}[PARAMS{Float64}([[0.5], [0.5]], [0.5, 0.5]), PARAMS{Float64}([[0.55], [0.55]], [0.45, 0.5]), PARAMS{Float64}([[0.6000000000000001], [0.6000000000000001]], [0.4, 0.5]), PARAMS{Float64}([[0.6500000000000001], [0.6500000000000001]], [0.35000000000000003, 0.5]), PARAMS{Float64}([[0.7000000000000002], [0.7000000000000002]], [0.30000000000000004, 0.5]), PARAMS{Float64}([[0.7500000000000002], [0.7500000000000002]], [0.35000000000000003, 0.5]), PARAMS{Float64}([[0.8000000000000003], [0.8000000000000003]], [0.30000000000000004, 0.45]), PARAMS{Float64}([[0.8500000000000003], [0.8500000000000003]], [0.35000000000000003, 0.5]), PARAMS{Float64}([[0.9000000000000004], [0.8000000000000003]], [0.30000000000000004, 0.5]), PARAMS{Float64}([[0.9500000000000004], [0.8500000000000003]], [0.35000000000000003, 0.5]), PARAMS{Float64}([[0.999999], [0.8000000000000003]], [0.30000000000000004, 0.55]), PARAMS{Float64}([[0.999999], [0.8500000000000003]], [0.35000000000000003, 0.5]), PARAM

201-element Vector{PARAMS{Float64}}:
 PARAMS{Float64}([[0.5], [0.5]], [0.5, 0.5])
 PARAMS{Float64}([[0.55], [0.55]], [0.45, 0.5])
 PARAMS{Float64}([[0.6000000000000001], [0.6000000000000001]], [0.4, 0.5])
 PARAMS{Float64}([[0.6500000000000001], [0.6500000000000001]], [0.35000000000000003, 0.5])
 PARAMS{Float64}([[0.7000000000000002], [0.7000000000000002]], [0.30000000000000004, 0.5])
 PARAMS{Float64}([[0.7500000000000002], [0.7500000000000002]], [0.35000000000000003, 0.5])
 PARAMS{Float64}([[0.8000000000000003], [0.8000000000000003]], [0.30000000000000004, 0.45])
 PARAMS{Float64}([[0.8500000000000003], [0.8500000000000003]], [0.35000000000000003, 0.5])
 PARAMS{Float64}([[0.9000000000000004], [0.8000000000000003]], [0.30000000000000004, 0.5])
 PARAMS{Float64}([[0.9500000000000004], [0.8500000000000003]], [0.35000000000000003, 0.5])
 PARAMS{Float64}([[0.999999], [0.8000000000000003]], [0.30000000000000004, 0.55])
 PARAMS{Float64}([[0.999999], [0.8500000000000003]], [0.35000000000000003, 